In [ ]:
import os
import zipfile
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zip_file_path = '/content/drive/MyDrive/kidney_refined_img.zip'
csv_file_path = '/content/drive/MyDrive/Coldsheet.csv'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/kidney_refined_img')

extracted_files = os.listdir('/content/drive/MyDrive/kidney_refined_img')

scores_df = pd.read_csv(csv_file_path)
scores_df.head()

,Name of image,Score of 1 - 5
0,Cold Kidney 2,5
1,Cold Kidney 3,5
2,Cold Kidney 4,5
3,Cold Kidney 5,5
4,Cold Kidney 6,4


# not working..column rename code

In [ ]:
scores_df.rename(columns = {'score':'Score 0f 1-5','NameofImage':'Name of image'}, inplace=True)

In [ ]:
scores_df.head()

,Name of image,Score of 1 - 5
0,Cold Kidney 2,5
1,Cold Kidney 3,5
2,Cold Kidney 4,5
3,Cold Kidney 5,5
4,Cold Kidney 6,4


# **Data Preparation**

In [ ]:
def load_and_preprocess_image(image_path):
    if os.path.isfile(image_path):
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        return img_array
    return None

In [ ]:
image_dir = '/content/drive/MyDrive/kidney_refined_img/images new'
image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, filename))]
image_arrays = [load_and_preprocess_image(path) for path in image_paths if load_and_preprocess_image(path) is not None]
image_arrays = np.vstack([img_array for img_array in image_arrays if img_array is not None])

In [ ]:
def format_filename(name):
    return os.path.splitext(name)[0].lower().strip()

In [ ]:
scores_df['formatted_name'] = scores_df['Name of image'].apply(format_filename)
# Map score according to images
name_to_score = dict(zip(scores_df['formatted_name'], scores_df['Score of 1 - 5']))

In [ ]:
# Cross the score with images and then create log file for those that is not matched
scores = []
not_found = []

for path in image_paths:
    if os.path.isfile(path):
        base_name = format_filename(os.path.basename(path))
        score = name_to_score.get(base_name, None)
        if score is not None:
            scores.append(score)
        else:
            not_found.append(os.path.basename(path))
            scores.append(-1)

if not_found:
    print("No scores found for the following files:")
    for file in not_found:
        print(file)
else:
    print("All files matched successfully.")

scores = np.array(scores)

All files matched successfully.


In [ ]:
valid_indices = [i for i, score in enumerate(scores) if score != -1]
image_arrays = image_arrays[valid_indices]
scores = np.array([scores[i] for i in valid_indices])

In [ ]:
base_model = tf.keras.applications.resnet50.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)


94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
base_model.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert scores to one-hot encoding
scores_one_hot = to_categorical(scores - 1, num_classes=5)
model.fit(image_arrays, scores_one_hot, epochs=10, batch_size=32, shuffle=True)

Epoch 1/10
4/4 [==============================] - 29s 6s/step - loss: 73.5690 - accuracy: 0.3400
Epoch 2/10
4/4 [==============================] - 24s 5s/step - loss: 12.4122 - accuracy: 0.7100
Epoch 3/10
4/4 [==============================] - 22s 5s/step - loss: 13.8890 - accuracy: 0.7500
Epoch 4/10
4/4 [==============================] - 24s 5s/step - loss: 4.4976 - accuracy: 0.9000
Epoch 5/10
4/4 [==============================] - 23s 5s/step - loss: 2.7758 - accuracy: 0.8700
Epoch 6/10
4/4 [==============================] - 23s 5s/step - loss: 0.0016 - accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 24s 5s/step - loss: 0.0017 - accuracy: 1.0000
Epoch 8/10
4/4 [==============================] - 22s 5s/step - loss: 0.1994 - accuracy: 0.9800
Epoch 9/10
4/4 [==============================] - 24s 5s/step - loss: 9.7867e-05 - accuracy: 1.0000
Epoch 10/10
4/4 [==============================] - 23s 6s/step - loss: 8.5995e-04 - accuracy: 1.0000


In [ ]:
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
resnet_model = model.get_layer('resnet50')
# Print all layer names
for layer in resnet_model.layers:
    print(layer.name)

In [ ]:
def load_and_preprocess_image(image_path):
    if os.path.isfile(image_path):
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        return img_array
    return None

In [ ]:
image_dir = '/content/drive/MyDrive/kidney_refined_img/images new'
image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, filename))]
image_arrays = [load_and_preprocess_image(path) for path in image_paths if load_and_preprocess_image(path) is not None]
image_arrays = np.vstack([img_array for img_array in image_arrays if img_array is not None])


# **training and testing model**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(image_arrays, scores, test_size=0.2, random_state=42)
y_train_one_hot = to_categorical(y_train - 1, num_classes=5)
y_test_one_hot = to_categorical(y_test - 1, num_classes=5)

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [ ]:
MyModel = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(5, activation='softmax')
])

MyModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

MyModel.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, shuffle = True)

Epoch 1/10
3/3 [==============================] - 22s 6s/step - loss: 57.7580 - accuracy: 0.2750
Epoch 2/10
3/3 [==============================] - 18s 5s/step - loss: 111.3902 - accuracy: 0.4375
Epoch 3/10
3/3 [==============================] - 19s 5s/step - loss: 24.2524 - accuracy: 0.7625
Epoch 4/10
3/3 [==============================] - 18s 6s/step - loss: 13.6784 - accuracy: 0.7500
Epoch 5/10
3/3 [==============================] - 18s 6s/step - loss: 6.5336 - accuracy: 0.8375
Epoch 6/10
3/3 [==============================] - 18s 5s/step - loss: 1.9573 - accuracy: 0.9500
Epoch 7/10
3/3 [==============================] - 19s 5s/step - loss: 0.8031 - accuracy: 0.9750
Epoch 8/10
3/3 [==============================] - 18s 6s/step - loss: 0.6378 - accuracy: 0.9750
Epoch 9/10
3/3 [==============================] - 27s 9s/step - loss: 0.2156 - accuracy: 0.9875
Epoch 10/10
3/3 [==============================] - 20s 6s/step - loss: 2.9802e-09 - accuracy: 1.0000


In [ ]:
# # Save the model
# model_path = "/content/drive/MyDrive/Model/MyModel"
# MyModel.save(model_path)


In [ ]:
# Saving training and test set

In [ ]:
# np.save('/content/drive/MyDrive/Model/X_train.npy', X_train)
# np.save('/content/drive/MyDrive/Model/X_test.npy', X_test)
# np.save('/content/drive/MyDrive/Model/y_train_one_hot.npy', y_train_one_hot)
# np.save('/content/drive/MyDrive/Model/y_test_one_hot.npy', y_test_one_hot)


In [ ]:
MyModel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_2 (Dense)             (None, 512)               51380736  
                                                                 
 dense_3 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 74971013 (285.99 MB)
Trainable params: 51383301 (196.01 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


# loading tarining and test set

In [ ]:
X_train = np.load('/content/drive/MyDrive/Model/X_train.npy')
X_test = np.load('/content/drive/MyDrive/Model/X_test.npy')
y_train_one_hot = np.load('/content/drive/MyDrive/Model/y_train_one_hot.npy')
y_test_one_hot = np.load('/content/drive/MyDrive/Model/y_test_one_hot.npy')


In [ ]:
predictions = MyModel.predict(X_test)
predicted_scores = np.argmax(predictions, axis=1) + 1

1/1 [==============================] - 5s 5s/step


# Checking model accuracy on test set

In [ ]:
# Convert probabilities to class labels; the predicted class is the one with the highest probability
y_pred = np.argmax(predictions, axis=1)

# Since y_test_one_hot is one-hot encoded, we need to convert it back to class labels
y_true = np.argmax(y_test_one_hot, axis=1)

# Calculate the accuracy
accuracy = accuracy_score(y_true, y_pred)

print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 95.00%


# loading the model

In [ ]:
# Load the model
loaded_model = load_model('/content/drive/MyDrive/MyModel')
# Now you can use loaded_model to make predictions, evaluate, etc.

# **Extract Features from Each Convolutional Layer**

In [ ]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [ ]:
# Load the base ResNet-50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False)

# Define the layers from which you want to extract features
layer_names = ['conv1_relu', 'conv2_block3_out', 'conv3_block4_out', 'conv4_block6_out', 'conv5_block3_out']
layer_outputs = [base_model.get_layer(name).output for name in layer_names]
multi_output_model = Model(inputs=base_model.input, outputs=layer_outputs)

def extract_features(image_tensor):
    # Expand the dimensions of the image tensor to include batch size
    image_tensor = np.expand_dims(image_tensor, axis=0)
    return multi_output_model.predict(image_tensor)

features_by_layer = [[] for _ in range(len(layer_names))]

for image_array in image_arrays:
    if image_array.ndim == 3:
        image_array = np.expand_dims(image_array, axis=0)

    if image_array.ndim != 4:
        print(f"Skipping an image due to unexpected dimensions: {image_array.shape}")
        continue

    # Extract features for all layers
    features_per_layer = extract_features(image_array[0])

    # Flatten and collect features separately for each layer
    for i, features in enumerate(features_per_layer):
        flattened_features = features.reshape(-1)  # Flatten the features
        features_by_layer[i].append(flattened_features)


pca_reduced_features_by_layer = []


for i, layer_features in enumerate(features_by_layer):

    features_array = np.vstack(layer_features)

    # number of PCA components (can be tuned or set dynamically)
    n_components = min(256, features_array.shape[0])

    pca = PCA(n_components=n_components)
    reduced_features = pca.fit_transform(features_array)

    pca_reduced_features_by_layer.append(reduced_features)

    # Save the reduced features to disk
    np.save(f'/content/drive/MyDrive/kidney_features_layer_{i}_reduced.npy', reduced_features)

    print(f"Layer {layer_names[i]}: original shape {features_array.shape}, reduced shape {reduced_features.shape}")


1/1 [==============================] - 0s 322ms/step
Layer conv1_relu: original shape (100, 802816), reduced shape (100, 100)
Layer conv2_block3_out: original shape (100, 802816), reduced shape (100, 100)
Layer conv3_block4_out: original shape (100, 401408), reduced shape (100, 100)
Layer conv4_block6_out: original shape (100, 200704), reduced shape (100, 100)
Layer conv5_block3_out: original shape (100, 100352), reduced shape (100, 100)


# **Testing features in 2D array**

In [ ]:

# Assuming image_arrays contains your preprocessed images (as loaded and preprocessed earlier)
# Initialize ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Iterate over each layer of the model to extract features
for i, layer in enumerate(base_model.layers):
    # Create a new model that will output the features from the current layer
    model = Model(inputs=base_model.inputs, outputs=layer.output)

    # Initialize an empty list to store the flattened features for each image
    flattened_features = []

    # Iterate over your preprocessed images and extract the features
    for image in image_arrays:
    # Ensure the image has a batch dimension
    # image should be of shape (1, 224, 224, 3)
      if image.ndim == 3:
        image = np.expand_dims(image, axis=0)

    # Check if the image shape is now correct (1, 224, 224, 3)
      assert image.shape == (1, 224, 224, 3), "Image shape is incorrect"

    # Get the features for one image
      features = model.predict(image)

    # Flatten the features and append to the list
      flattened_features.append(features.flatten())


    # Combine the flattened features from all images into a 2D array
    features_array = np.array(flattened_features)

    # Ensure the shape is what you expect and save the array
    print(f"Shape for layer {i}: {features_array.shape}")
    # Save the features for this layer to disk
    np.save(f'features_layer_{i}.npy', features_array)


In [ ]:
# all_features_array.shape

(100, 2308096)

# **Combine and Prepare the Extracted Features**
- Matching target with features

In [ ]:
# Assuming 'pca_reduced_features_by_layer[-1]' contains the PCA reduced features from the last layer 'conv5_block3_out'
features = pca_reduced_features_by_layer[-1]

# Assuming 'image_paths' contains the paths for each image corresponding to each feature row in 'features'
# 'name_to_score' is a dictionary mapping filenames to scores

# Match scores with features
matched_scores = []
feature_list = []
not_found = []

for i, path in enumerate(image_paths):
    if os.path.isfile(path):
        base_name = format_filename(os.path.basename(path))
        score = name_to_score.get(base_name, None)
        if score is not None:
            matched_scores.append(score)
            feature_list.append(features[i])
        else:
            not_found.append(os.path.basename(path))

if not_found:
    print("No scores found for the following files:")
    for file in not_found:
        print(file)
else:
    print("All files matched successfully.")

# Convert lists to numpy arrays for machine learning processing
matched_scores = np.array(matched_scores)
feature_array = np.vstack(feature_list)


All files matched successfully.


# **L2 norm**

In [ ]:
import numpy as np
from numpy.linalg import norm

# Assuming features_by_layer contains the extracted flattened features for each layer

l2_norms_by_layer = []

for layer_features in features_by_layer:
    # Stack the layer's features if they're in a list of arrays
    features_array = np.vstack(layer_features)

    # Compute the L2 norm for each feature set within the current layer
    l2_norms = np.linalg.norm(features_array, ord=2, axis=1)

    # Store the computed L2 norms for the current layer
    l2_norms_by_layer.append(l2_norms)

    # Optionally, you can save the L2 norms to disk if needed
    # np.save(f'/content/drive/MyDrive/kidney_features_layer_{layer_index}_l2_norms.npy', l2_norms)


# **Classifier** for 5th layer only

In [ ]:

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(feature_array, matched_scores, test_size=0.2, random_state=42)

# Initialize and train the classifier
classifier = LogisticRegression(max_iter=1000)  # Increasing max_iter for convergence if necessary
classifier.fit(X_train, y_train)

# Predict and evaluate the classifier
predictions = classifier.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

print(f"Test Accuracy: {accuracy:.2f}")


Test Accuracy: 0.55


# classifier for all 5 layers

In [ ]:
# Function to train and evaluate a classifier for given features and scores
def train_and_evaluate(features, scores, layer_name):
    X_train, X_test, y_train, y_test = train_test_split(features, scores, test_size=0.2, random_state=42)

    classifier = LogisticRegression(max_iter=1000)
    classifier.fit(X_train, y_train)

    predictions = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)

    print(f"Accuracy for {layer_name}: {accuracy:.2f}")

# Assuming 'pca_reduced_features_by_layer' contains the PCA-reduced features for all 5 layers
# Assuming 'scores' contains the matched target scores for each image
for i, layer_features in enumerate(pca_reduced_features_by_layer):
    layer_name = layer_names[i]  # Assuming 'layer_names' list is defined as in previous examples
    train_and_evaluate(layer_features, scores, layer_name)

Accuracy for conv1_relu: 0.40
Accuracy for conv2_block3_out: 0.40
Accuracy for conv3_block4_out: 0.45
Accuracy for conv4_block6_out: 0.35
Accuracy for conv5_block3_out: 0.55


# **Training a Classifier on the Combined Features**

In [ ]:
# Calculate accuracy and confusion matrix
#accuracy = accuracy_score(y_test, predicted_scores)
#conf_matrix = confusion_matrix(y_test, predicted_scores)

#print(f"Test Accuracy: {accuracy}")
#print(f"Confusion Matrix:\n{conf_matrix}")